In [3]:
%pip install numpy pandas tqdm scikit-learn matplotlib seaborn ucimlrepo


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [25]:
from ucimlrepo import fetch_ucirepo 
import numpy as np
import pandas as pd

steel_industry_energy_consumption = fetch_ucirepo(id=851) 
  
X = steel_industry_energy_consumption.data.features 
y = steel_industry_energy_consumption.data.targets 
  
day_mapping = {'Monday': 0, 'Tuesday': 1, 'Wednesday': 2, 'Thursday': 3, 'Friday': 4, 'Saturday': 5, 'Sunday': 6}
X['Day_of_week'] = X['Day_of_week'].map(day_mapping)
X['Day_sin'] = np.sin(2 * np.pi * X['Day_of_week'] / 7)
X['Day_cos'] = np.cos(2 * np.pi * X['Day_of_week'] / 7)
X = pd.get_dummies(X, columns=['WeekStatus'])
y = pd.get_dummies(y, columns=["Load_Type"])


/var/folders/2k/j64djv5131g0m2v23qm8g2lm0000gn/T/ipykernel_96738/1992521999.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Day_of_week'] = X['Day_of_week'].map(day_mapping)
/var/folders/2k/j64djv5131g0m2v23qm8g2lm0000gn/T/ipykernel_96738/1992521999.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Day_sin'] = np.sin(2 * np.pi * X['Day_of_week'] / 7)


In [26]:
X

,Usage_kWh,Lagging_Current_Reactive.Power_kVarh,Leading_Current_Reactive_Power_kVarh,CO2(tCO2),Lagging_Current_Power_Factor,Leading_Current_Power_Factor,NSM,Day_of_week,Day_sin,Day_cos,WeekStatus_Weekday,WeekStatus_Weekend
0,3.17,2.95,0.00,0.0,73.21,100.00,900,0,0.0,1.0,True,False
1,4.00,4.46,0.00,0.0,66.77,100.00,1800,0,0.0,1.0,True,False
2,3.24,3.28,0.00,0.0,70.28,100.00,2700,0,0.0,1.0,True,False
3,3.31,3.56,0.00,0.0,68.09,100.00,3600,0,0.0,1.0,True,False
4,3.82,4.50,0.00,0.0,64.72,100.00,4500,0,0.0,1.0,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...
35035,3.85,4.86,0.00,0.0,62.10,100.00,82800,0,0.0,1.0,True,False
35036,3.74,3.74,0.00,0.0,70.71,100.00,83700,0,0.0,1.0,True,False
35037,3.78,3.17,0.07,0.0,76.62,99.98,84600,0,0.0,1.0,True,False
35038,3.78,3.06,0.11,0.0,77.72,99.96,85500,0,0.0,1.0,True,False


In [33]:
y

,Load_Type_Light_Load,Load_Type_Maximum_Load,Load_Type_Medium_Load
0,True,False,False
1,True,False,False
2,True,False,False
3,True,False,False
4,True,False,False
...,...,...,...
35035,True,False,False
35036,True,False,False
35037,True,False,False
35038,True,False,False


In [29]:
X_all = X.to_numpy(dtype=np.float32)
y_all = y.to_numpy(dtype=np.int8)

In [32]:
(X_all.shape, y_all.shape)

((35040, 12), (35040, 3))

In [34]:
split_idx = int(0.8 * len(X_all))

X_train = X_all[:split_idx]
X_test = X_all[split_idx:]
y_train = y_all[:split_idx]
y_test = y_all[split_idx:]